In [1]:
import argparse
import requests
import numpy as np
import pandas as pd
import hail as hl
from hail.expr.functions import _sort_by
from hail.utils import hadoop_open, new_temp_file
import pyranges as pr
from ukbb_pan_ancestry import (
    get_distance_clumping_results_path,
    get_known_ukbb_loci_path,
    get_meta_analysis_results_path,
    get_munged_otg_v2d_path,
    get_munged_round2_path,
    get_pheno_manifest_path,
    get_ukb_pheno_efo_mapping_path,
    load_final_sumstats_mt,
    otg_release,
)

Loading BokehJS ...

In [19]:
#--------------------------- 
# 09/20/2024
# expand 'region' to within 1Mb of lead variants
#---------------------------

In [2]:
def _to_pandas(ht):
            df = ht.to_pandas()
            # a temporary fix: cf. https://github.com/biocore-ntnu/pyranges/pull/264
            df = df.astype({k: "object" if v == "string[python]" else str(v).lower() for k, v in df.dtypes.items()})
            return df

In [8]:
# otg lead variants
#ht_v2d = hl.read_table("/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/munged_otg_v2d_no_exclusion.ht")
#ht_v2d = ht_v2d.annotate(
#    study_id=hl.delimit(ht_v2d.study_id),
#    Chromosome=ht_v2d.locus.contig,
#    Start=ht_v2d.locus.position,
#    End=ht_v2d.locus.position,
#)
#df_v2d = _to_pandas(ht_v2d)
print(df_v2d.head())
#gr_v2d = pr.PyRanges(df_v2d)
print(gr_v2d.head())

      locus    trait_efo           study_id                    trait_efo_term  \
0  1:768253  EFO_0006312       GCST90026371     mitochondrial DNA measurement   
1  1:839538  EFO_0006865  NEALE2_20002_1202      urgency urinary incontinence   
2  1:858051  EFO_0004617       GCST90025945            cystatin c measurement   
3  1:891059  EFO_0004533       GCST90019494  alkaline phosphatase measurement   
4  1:894573  EFO_0002618         GCST005434              pancreatic carcinoma   

         trait_efo_category Chromosome   Start     End  
0         Other measurement          1  768253  768253  
1               Other trait          1  839538  839538  
2         Other measurement          1  858051  858051  
3  Liver enzyme measurement          1  891059  891059  
4                    Cancer          1  894573  894573  
+------------+-------------+----------------------------------+-------+
| locus      | trait_efo   | study_id                         | +5    |
| (object)   | (object)    

In [9]:
trait = "ALCO_AMOUNT"
# kcps2 lead variants
df = pd.read_table(f"/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/KCPS2_loci/GenomicRiskLoci_{trait}.txt", delimiter="\t")
print(df)

# pyrange
gr = pr.PyRanges(df)
print(gr)

   idx    lead_locus                  region    trait_efo trait_efo_category  \
0    1    3:38052725     3:38052725-38052725  EFO_0007878  Other measurement   
1    2   4:100239319   4:100000136-100336448  EFO_0007878  Other measurement   
2    3    6:26207175     6:26180634-26326182  EFO_0007878  Other measurement   
3    4  12:108700164  12:108248685-108701635  EFO_0007878  Other measurement   
4    5  12:112241766  12:111414461-113117897  EFO_0007878  Other measurement   

   Chromosome      Start        End  
0           3   38052725   38052725  
1           4  100000136  100336448  
2           6   26180634   26326182  
3          12  108248685  108701635  
4          12  111414461  113117897  
+-----------+--------------+------------------------+-------------+-------+
|       idx | lead_locus   | region                 | trait_efo   | +4    |
|   (int64) | (object)     | (object)               | (object)    | ...   |
|-----------+--------------+------------------------+----------

/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [10]:
gr['4', 100000136:100336445]

,idx,lead_locus,region,trait_efo,trait_efo_category,Chromosome,Start,End
1,2,4:100239319,4:100000136-100336448,EFO_0007878,Other measurement,4,100000136,100336448


In [12]:
df_annotated = gr.join(gr_v2d, how="left", suffix="_otg").as_df()
#df_annotated = df_annotated.drop(columns=["Chromosome", "Start", "End", "Start_otg", "End_otg"])
print(df_annotated)

      idx    lead_locus                  region    trait_efo  \
0       1    3:38052725     3:38052725-38052725  EFO_0007878   
1       2   4:100239319   4:100000136-100336448  EFO_0007878   
2       2   4:100239319   4:100000136-100336448  EFO_0007878   
3       2   4:100239319   4:100000136-100336448  EFO_0007878   
4       2   4:100239319   4:100000136-100336448  EFO_0007878   
...   ...           ...                     ...          ...   
1259    5  12:112241766  12:111414461-113117897  EFO_0007878   
1260    5  12:112241766  12:111414461-113117897  EFO_0007878   
1261    5  12:112241766  12:111414461-113117897  EFO_0007878   
1262    5  12:112241766  12:111414461-113117897  EFO_0007878   
1263    5  12:112241766  12:111414461-113117897  EFO_0007878   

     trait_efo_category Chromosome      Start        End         locus  \
0     Other measurement          3   38052725   38052725            -1   
1     Other measurement          4  100000136  100336448   4:100006645   
2     Oth

In [11]:
#== test ==#
# otg lead variants
ht_v2d = hl.read_table("/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/munged_otg_v2d_no_exclusion.ht")
ht_v2d = ht_v2d.annotate(
    study_id=hl.delimit(ht_v2d.study_id),
    Chromosome=ht_v2d.locus.contig,
    Start=ht_v2d.locus.position,
    End=ht_v2d.locus.position,
)
df_v2d = _to_pandas(ht_v2d)
print(df_v2d.head())

# Export to TSV or CSV
ht_v2d.export('/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/munged_otg_v2d_no_exclusion.tsv')

      locus    trait_efo           study_id                    trait_efo_term  \
0  1:768253  EFO_0006312       GCST90026371     mitochondrial DNA measurement   
1  1:839538  EFO_0006865  NEALE2_20002_1202      urgency urinary incontinence   
2  1:858051  EFO_0004617       GCST90025945            cystatin c measurement   
3  1:891059  EFO_0004533       GCST90019494  alkaline phosphatase measurement   
4  1:894573  EFO_0002618         GCST005434              pancreatic carcinoma   

         trait_efo_category Chromosome   Start     End  
0         Other measurement          1  768253  768253  
1               Other trait          1  839538  839538  
2         Other measurement          1  858051  858051  
3  Liver enzyme measurement          1  891059  891059  
4                    Cancer          1  894573  894573  


In [7]:
def annotate_known_loci(trait):

    # otg lead variants
    ht_v2d = hl.read_table("/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/munged_otg_v2d_no_exclusion.ht")
    ht_v2d = ht_v2d.annotate(
        study_id=hl.delimit(ht_v2d.study_id),
        Chromosome=ht_v2d.locus.contig,
        Start=ht_v2d.locus.position,
        End=ht_v2d.locus.position,
    )
    df_v2d = _to_pandas(ht_v2d)
    #print(df_v2d.head())

    # kcps2 lead variants
    df = pd.read_table(f"/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/KCPS2_loci/GenomicRiskLoci_{trait}.txt", delimiter="\t")
    #print(df.head())
    
    # pyrange
    gr = pr.PyRanges(df)
    #print(gr.head())

    gr_v2d = pr.PyRanges(df_v2d)
    #print(gr_v2d.head())

    df_annotated = gr.join(gr_v2d, how="left", suffix="_otg").as_df()
    df_annotated = df_annotated.drop(columns=["Chromosome", "Start", "End", "Start_otg", "End_otg"])

    # write out and read back b/c hl.Table.from_pandas is extreamly slow
    tmpfile = new_temp_file()
    with hadoop_open(tmpfile, "wb") as f:
        df_annotated.to_csv(f, sep="\t", na_rep="NA", index=False)

    ht = hl.import_table(tmpfile, impute=True, min_partitions=1000)
    ht = ht.rename({"locus": "locus_otg", "study_id": "study_id_otg"})
    ht = ht.annotate(
        **{
            key: hl.if_else(ht[key] == "-1", hl.missing(hl.tstr), ht[key])
            for key in filter(lambda x: x.endswith("_otg"), list(ht.row))
        }
    )
    ht = ht.annotate(
        lead_locus=hl.parse_locus(ht.lead_locus),
        region=hl.parse_locus_interval(ht.region),
        locus=hl.parse_locus(ht.locus_otg),
        study_id_otg=ht.study_id_otg.split(","),
    )
    ht.show(3)

    for field in ["trait_efo", "trait_efo_category"]:
        x = df_annotated[~df_annotated[field].isna()]
        known_idx = set(x[x[field] == x[f"{field}_otg"]].idx)
        novel_idx = set(x.idx) - set(known_idx)
        na_idx = set(df_annotated.idx) - set(x.idx)
        na_match = na_idx.intersection(
            set(df_annotated[df_annotated[field].isna() & ~df_annotated.locus.isna()].idx)
        )
        k = "efo" if field == "trait_efo" else "efo_category"
        ht = ht.annotate(
            **{
                f"is_known_{k}": hl.case()
                #.when(hl.set(known_idx).contains(ht.idx), True)
                #.when(hl.set(novel_idx).contains(ht.idx), False)
                .when(hl.set(known_idx if len(known_idx) > 0 else hl.empty_set(ht.idx.dtype)).contains(ht.idx), True)
                .when(hl.set(novel_idx if len(novel_idx) > 0 else hl.empty_set(ht.idx.dtype)).contains(ht.idx), False)
                .default(hl.missing(hl.tbool))
            }
        )
        print(field)
        # category known
        print(len(known_idx))
        # category novel
        print(len(novel_idx))
        # category NA
        print(len(na_idx))
        # category NA match
        print(len(na_match))
        
    #N_novel = len(novel_idx)
    #N_novel_cat = len(novel_idx)
    #res_count = pd.DataFrame({'trait': [trait], 'Number of loci': [N_total], 'Number of novel loci': [N_unrep]})
    
    ht.export(f"/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/KCPS2_otg/known_kcps2_loci_{trait}.tsv")


In [4]:
def annotate_known_loci_1Mb(trait):

    # otg lead variants
    ht_v2d = hl.read_table("/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/munged_otg_v2d_no_exclusion.ht")
    ht_v2d = ht_v2d.annotate(
        study_id=hl.delimit(ht_v2d.study_id),
        Chromosome=ht_v2d.locus.contig,
        Start=ht_v2d.locus.position,
        End=ht_v2d.locus.position,
    )
    df_v2d = _to_pandas(ht_v2d)
    #print(df_v2d.head())

    # kcps2 lead variants
    df = pd.read_table(f"/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/KCPS2_loci/GenomicRiskLoci_{trait}_1Mb.txt", delimiter="\t")
    #print(df.head())
    
    # pyrange
    gr = pr.PyRanges(df)
    #print(gr.head())

    gr_v2d = pr.PyRanges(df_v2d)
    #print(gr_v2d.head())

    df_annotated = gr.join(gr_v2d, how="left", suffix="_otg").as_df()
    df_annotated = df_annotated.drop(columns=["Chromosome", "Start", "End", "Start_otg", "End_otg"])

    # write out and read back b/c hl.Table.from_pandas is extreamly slow
    tmpfile = new_temp_file()
    with hadoop_open(tmpfile, "wb") as f:
        df_annotated.to_csv(f, sep="\t", na_rep="NA", index=False)

    ht = hl.import_table(tmpfile, impute=True, min_partitions=1000)
    ht = ht.rename({"locus": "locus_otg", "study_id": "study_id_otg"})
    ht = ht.annotate(
        **{
            key: hl.if_else(ht[key] == "-1", hl.missing(hl.tstr), ht[key])
            for key in filter(lambda x: x.endswith("_otg"), list(ht.row))
        }
    )
    ht = ht.annotate(
        lead_locus=hl.parse_locus(ht.lead_locus),
        region=hl.parse_locus_interval(ht.region),
        locus=hl.parse_locus(ht.locus_otg),
        study_id_otg=ht.study_id_otg.split(","),
    )
    ht.show(3)

    for field in ["trait_efo", "trait_efo_category"]:
        x = df_annotated[~df_annotated[field].isna()]
        known_idx = set(x[x[field] == x[f"{field}_otg"]].idx)
        novel_idx = set(x.idx) - set(known_idx)
        na_idx = set(df_annotated.idx) - set(x.idx)
        na_match = na_idx.intersection(
            set(df_annotated[df_annotated[field].isna() & ~df_annotated.locus.isna()].idx)
        )
        k = "efo" if field == "trait_efo" else "efo_category"
        ht = ht.annotate(
            **{
                f"is_known_{k}": hl.case()
                #.when(hl.set(known_idx).contains(ht.idx), True)
                #.when(hl.set(novel_idx).contains(ht.idx), False)
                .when(hl.set(known_idx if len(known_idx) > 0 else hl.empty_set(ht.idx.dtype)).contains(ht.idx), True)
                .when(hl.set(novel_idx if len(novel_idx) > 0 else hl.empty_set(ht.idx.dtype)).contains(ht.idx), False)
                .default(hl.missing(hl.tbool))
            }
        )
        print(field)
        # category known
        print(len(known_idx))
        # category novel
        print(len(novel_idx))
        # category NA
        print(len(na_idx))
        # category NA match
        print(len(na_match))
        
    #N_novel = len(novel_idx)
    #N_novel_cat = len(novel_idx)
    #res_count = pd.DataFrame({'trait': [trait], 'Number of loci': [N_total], 'Number of novel loci': [N_unrep]})
    
    ht.export(f"/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/KCPS2_otg/known_kcps2_loci_{trait}_1Mb.tsv")


In [3]:
def annotate_known_loci_500kb(trait):

    # otg lead variants
    ht_v2d = hl.read_table("/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/munged_otg_v2d_no_exclusion.ht")
    ht_v2d = ht_v2d.annotate(
        study_id=hl.delimit(ht_v2d.study_id),
        Chromosome=ht_v2d.locus.contig,
        Start=ht_v2d.locus.position,
        End=ht_v2d.locus.position,
    )
    df_v2d = _to_pandas(ht_v2d)
    #print(df_v2d.head())

    # kcps2 lead variants
    df = pd.read_table(f"/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/KCPS2_loci/GenomicRiskLoci_{trait}_500kb.txt", delimiter="\t")
    #print(df.head())
    
    # pyrange
    gr = pr.PyRanges(df)
    #print(gr.head())

    gr_v2d = pr.PyRanges(df_v2d)
    #print(gr_v2d.head())

    df_annotated = gr.join(gr_v2d, how="left", suffix="_otg").as_df()
    df_annotated = df_annotated.drop(columns=["Chromosome", "Start", "End", "Start_otg", "End_otg"])

    # write out and read back b/c hl.Table.from_pandas is extreamly slow
    tmpfile = new_temp_file()
    with hadoop_open(tmpfile, "wb") as f:
        df_annotated.to_csv(f, sep="\t", na_rep="NA", index=False)

    ht = hl.import_table(tmpfile, impute=True, min_partitions=1000)
    ht = ht.rename({"locus": "locus_otg", "study_id": "study_id_otg"})
    ht = ht.annotate(
        **{
            key: hl.if_else(ht[key] == "-1", hl.missing(hl.tstr), ht[key])
            for key in filter(lambda x: x.endswith("_otg"), list(ht.row))
        }
    )
    ht = ht.annotate(
        lead_locus=hl.parse_locus(ht.lead_locus),
        region=hl.parse_locus_interval(ht.region),
        locus=hl.parse_locus(ht.locus_otg),
        study_id_otg=ht.study_id_otg.split(","),
    )
    ht.show(3)

    for field in ["trait_efo", "trait_efo_category"]:
        x = df_annotated[~df_annotated[field].isna()]
        known_idx = set(x[x[field] == x[f"{field}_otg"]].idx)
        novel_idx = set(x.idx) - set(known_idx)
        na_idx = set(df_annotated.idx) - set(x.idx)
        na_match = na_idx.intersection(
            set(df_annotated[df_annotated[field].isna() & ~df_annotated.locus.isna()].idx)
        )
        k = "efo" if field == "trait_efo" else "efo_category"
        ht = ht.annotate(
            **{
                f"is_known_{k}": hl.case()
                #.when(hl.set(known_idx).contains(ht.idx), True)
                #.when(hl.set(novel_idx).contains(ht.idx), False)
                .when(hl.set(known_idx if len(known_idx) > 0 else hl.empty_set(ht.idx.dtype)).contains(ht.idx), True)
                .when(hl.set(novel_idx if len(novel_idx) > 0 else hl.empty_set(ht.idx.dtype)).contains(ht.idx), False)
                .default(hl.missing(hl.tbool))
            }
        )
        print(field)
        # category known
        print(len(known_idx))
        # category novel
        print(len(novel_idx))
        # category NA
        print(len(na_idx))
        # category NA match
        print(len(na_match))
        
    #N_novel = len(novel_idx)
    #N_novel_cat = len(novel_idx)
    #res_count = pd.DataFrame({'trait': [trait], 'Number of loci': [N_total], 'Number of novel loci': [N_unrep]})
    
    ht.export(f"/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/KCPS2_otg/known_kcps2_loci_{trait}_500kb.tsv")


In [ ]:
# 1Mb

In [6]:
trait_list = ["LDL", "TG", "HDL", "CHO", "FBS", "INSULIN","GOT", "GPT", "GGT", "BIL", "ALB","TSH", "CEA","CREAT","ADIPO","URIC", "ALP","HB", "HCT", "MCH", "MCHC", "MCV","RBC", "RDW", "WBC","PLT", "EOS", "SBP", "DBP", "WT",
              "HEIGHT", "BMI", "WAIST", "SMOKA_MOD", "ALCO_AMOUNT","COFFA"]
#trait_list = ["LDL", "TG", "HDL", "CHO", "FBS", "GOT", "GPT", "GGT", "BIL", "ALB","TSH", "CEA","CREAT","ADIPO","URIC", "ALP","HB", "HCT", "MCH", "MCV","RBC", "RDW", "WBC","PLT", "EOS", "SBP", "DBP", "WT",
#              "HEIGHT", "BMI", "WAIST", "ALCO_AMOUNT"]

#to run efo only: "HDL","FBS", "BIL", "ALB","TSH", "CEA","CREAT","ADIPO","ALP","HCT","MCV","RDW", "WBC","EOS", "SBP", "DBP", "WT","BMI", , "ALCO_AMOUNT"
res = pd.DataFrame()
for i in range(len(trait_list)):
    trait = trait_list[i]
    print(trait)
    annotate_known_loci_1Mb(trait)    

Initializing Hail with default parameters...


LDL
24/09/21 13:44:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.4
SparkUI available at http://holy7c04106.rc.fas.harvard.edu:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.130-bea04d9c79b5
LOGGING: writing to /n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/hail-20240921-1344-0.2.130-bea04d9c79b5.log
/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of 

,,,,,,,,,,
idx,lead_locus,region,trait_efo,trait_efo_category,locus_otg,trait_efo_otg,study_id_otg,trait_efo_term,trait_efo_category_otg,locus
int32,locus<GRCh37>,interval<locus<GRCh37>>,str,str,str,str,array<str>,str,str,locus<GRCh37>
1,1:16505320,[1:15505320-1:17505320),"""EFO_0004611""","""Lipid or lipoprotein measurement""","""1:15508945""","""EFO_0009795""","[""GCST90025991""]","""serum urea measurement""","""Other measurement""",1:15508945
1,1:16505320,[1:15505320-1:17505320),"""EFO_0004611""","""Lipid or lipoprotein measurement""","""1:15510811""","""EFO_0009795""","[""GCST90019525""]","""serum urea measurement""","""Other measurement""",1:15510811
1,1:16505320,[1:15505320-1:17505320),"""EFO_0004611""","""Lipid or lipoprotein measurement""","""1:15538296""","""MONDO_0005277""","[""NEALE2_20002_1265""]","""migraine disorder""","""Cardiovascular disease""",1:15538296


trait_efo
98
10
0
0
trait_efo_category
104
4
0
0


...

COFFA


/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

2024-09-21 13:57:23.790 Hail: INFO: Reading table to impute column types
2024-09-21 13:57:28.846 Hail: INFO: Finished type imputation>  (951 + 2) / 1000]
  Loading field 'idx' as type int32 (imputed)
  Loading field 'lead_locus' as type str (imputed)
  Loading field 'region' as type str (imputed)
  Loading field 'trait_efo' as type 

,,,,,,,,,,
idx,lead_locus,region,trait_efo,trait_efo_category,locus_otg,trait_efo_otg,study_id_otg,trait_efo_term,trait_efo_category_otg,locus
int32,locus<GRCh37>,interval<locus<GRCh37>>,str,str,str,str,array<str>,str,str,locus<GRCh37>
1,4:89020503,[4:88020503-4:90020503),"""EFO_0006781""","""Other measurement""","""4:88022709""","""EFO_0004329""","[""GCST008083_3"",""GCST008083_4"",""GCST008083"",""GCST008084_4"",""GCST008084""]","""alcohol drinking""","""Biological process""",4:88022709
1,4:89020503,[4:88020503-4:90020503),"""EFO_0006781""","""Other measurement""","""4:88022709""","""EFO_0004530""","[""GCST008076_3"",""GCST008074_3"",""GCST008087"",""GCST008076"",""GCST008087_4"",""GCST008074_4"",""GCST008083_3"",""GCST008083_4"",""GCST008083"",""GCST008087_3"",""GCST008074""]","""triglyceride measurement""","""Lipid or lipoprotein measurement""",4:88022709
1,4:89020503,[4:88020503-4:90020503),"""EFO_0006781""","""Other measurement""","""4:88022709""","""EFO_0004584""","[""GCST90002395""]","""mean platelet volume""","""Hematological measurement""",4:88022709


trait_efo
3
0
0
0
trait_efo_category
3
0
0
0


2024-09-21 13:57:35.146 Hail: INFO: merging 1001 files totalling 289.0K... 1000]
2024-09-21 13:57:35.242 Hail: INFO: while writing:
    /n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/KCPS2_otg/known_kcps2_loci_COFFA_1Mb.tsv
  merge time: 95.475ms
2024-09-21 14:29:52.596 Hail: INFO: Reading table to impute column types
2024-09-21 14:29:58.318 Hail: INFO: Finished type imputation
  Loading field 'idx' as type int32 (imputed)
  Loading field 'lead_locus' as type str (imputed)
  Loading field 'region' as type str (imputed)
  Loading field 'trait_efo' as type str (imputed)
  Loading field 'trait_efo_category' as type str (imputed)
  Loading field 'locus' as type str (imputed)
  Loading field 'trait_efo_otg' as type str (imputed)
  Loading field 'study_id' as type str (imputed)
  Loading field 'trait_efo_term' as type str (imputed)
  Loading field 'trait_efo_category_otg' as type str (imputed)
2024-09-21 14:30:05.924 Hail: INFO: merging 1001 files totalling 4.4M...
2024-09-21 14:30:06.034 Hail: I

In [ ]:
# 500kb

In [10]:
trait_list = ["LDL", "TG", "HDL", "CHO", "FBS", "INSULIN","GOT", "GPT", "GGT", "BIL", "ALB","TSH", "CEA","CREAT","ADIPO","URIC", "ALP","HB", "HCT", "MCH", "MCHC", "MCV","RBC", "RDW", "WBC","PLT", "EOS", "SBP", "DBP", "WT",
              "HEIGHT", "BMI", "WAIST", "SMOKA_MOD", "ALCO_AMOUNT","COFFA"]
#trait_list = ["LDL", "TG", "HDL", "CHO", "FBS", "GOT", "GPT", "GGT", "BIL", "ALB","TSH", "CEA","CREAT","ADIPO","URIC", "ALP","HB", "HCT", "MCH", "MCV","RBC", "RDW", "WBC","PLT", "EOS", "SBP", "DBP", "WT",
#              "HEIGHT", "BMI", "WAIST", "ALCO_AMOUNT"]

#to run efo only: "HDL","FBS", "BIL", "ALB","TSH", "CEA","CREAT","ADIPO","ALP","HCT","MCV","RDW", "WBC","EOS", "SBP", "DBP", "WT","BMI", , "ALCO_AMOUNT"
res = pd.DataFrame()
for i in range(len(trait_list)):
    trait = trait_list[i]
    print(trait)
    annotate_known_loci_500kb(trait)    

LDL


/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,,,,,,,,,,
idx,lead_locus,region,trait_efo,trait_efo_category,locus_otg,trait_efo_otg,study_id_otg,trait_efo_term,trait_efo_category_otg,locus
int32,locus<GRCh37>,interval<locus<GRCh37>>,str,str,str,str,array<str>,str,str,locus<GRCh37>
1,1:16505320,[1:16005320-1:17005320),"""EFO_0004611""","""Lipid or lipoprotein measurement""","""1:16006604""","""EFO_0004464""","[""GCST90095129""]","""brain measurement""","""Other measurement""",1:16006604
1,1:16505320,[1:16005320-1:17005320),"""EFO_0004611""","""Lipid or lipoprotein measurement""","""1:16042766""","""EFO_0005763""","[""GCST90000061""]","""pulse pressure measurement""","""Cardiovascular measurement""",1:16042766
1,1:16505320,[1:16005320-1:17005320),"""EFO_0004611""","""Lipid or lipoprotein measurement""","""1:16042766""","""EFO_0006335""","[""GCST90000062""]","""systolic blood pressure""","""Cardiovascular measurement""",1:16042766


trait_efo
95
13
0
0
trait_efo_category
102
6
0
0


TG


...

trait_efo
5
0
0
0
trait_efo_category
5
0
0
0


COFFA


/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,,,,,,,,,,
idx,lead_locus,region,trait_efo,trait_efo_category,locus_otg,trait_efo_otg,study_id_otg,trait_efo_term,trait_efo_category_otg,locus
int32,locus<GRCh37>,interval<locus<GRCh37>>,str,str,str,str,array<str>,str,str,locus<GRCh37>
1,4:89020503,[4:88520503-4:89520503),"""EFO_0006781""","""Other measurement""","""4:88555608""","""EFO_0011011""","[""NEALE2_20015_raw""]","""sitting height measurement""","""Body measurement""",4:88555608
1,4:89020503,[4:88520503-4:89520503),"""EFO_0006781""","""Other measurement""","""4:88557753""","""EFO_0004980""","[""GCST90000025"",""GCST90000027""]","""appendicular lean mass""","""Other measurement""",4:88557753
1,4:89020503,[4:88520503-4:89520503),"""EFO_0006781""","""Other measurement""","""4:88557753""","""EFO_0004995""","[""NEALE2_23121_raw"",""NEALE2_23125_raw"",""NEALE2_23129_raw"",""NEALE2_23122_raw"",""NEALE2_23114_raw""]","""lean body mass""","""Other measurement""",4:88557753


trait_efo
3
0
0
0
trait_efo_category
3
0
0
0


In [ ]:
# 10/05/2024
# After correcting EFO_ID for FBS

In [5]:
trait_list = ["FBS"]
res = pd.DataFrame()
for i in range(len(trait_list)):
    trait = trait_list[i]
    print(trait)
    annotate_known_loci_500kb(trait)  

Initializing Hail with default parameters...


FBS
24/10/06 00:00:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.4
SparkUI available at http://holy7c04105.rc.fas.harvard.edu:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.130-bea04d9c79b5
LOGGING: writing to /n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/hail-20241006-0000-0.2.130-bea04d9c79b5.log
/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of 

,,,,,,,,,,
idx,lead_locus,region,trait_efo,trait_efo_category,locus_otg,trait_efo_otg,study_id_otg,trait_efo_term,trait_efo_category_otg,locus
int32,locus<GRCh37>,interval<locus<GRCh37>>,str,str,str,str,array<str>,str,str,locus<GRCh37>
1,1:43453188,[1:42953188-1:43953188),"""EFO_0004465""","""Other measurement""","""1:43001755""","""EFO_0005763""","[""GCST007269""]","""pulse pressure measurement""","""Cardiovascular measurement""",1:43001755
1,1:43453188,[1:42953188-1:43953188),"""EFO_0004465""","""Other measurement""","""1:43037556""","""EFO_0005763""","[""GCST006629""]","""pulse pressure measurement""","""Cardiovascular measurement""",1:43037556
1,1:43453188,[1:42953188-1:43953188),"""EFO_0004465""","""Other measurement""","""1:43066594""","""EFO_0006335""","[""GCST007267""]","""systolic blood pressure""","""Cardiovascular measurement""",1:43066594


trait_efo
30
57
0
0
trait_efo_category
87
0
0
0


2024-10-06 00:02:01.890 Hail: INFO: merging 1001 files totalling 2.9M... / 1000]
2024-10-06 00:02:02.071 Hail: INFO: while writing:
    /n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/KCPS2_otg/known_kcps2_loci_FBS_500kb.tsv
  merge time: 179.050ms
2024-10-06 00:08:47.353 Hail: INFO: Reading table to impute column types
2024-10-06 00:08:55.068 Hail: INFO: Finished type imputation
  Loading field 'idx' as type int32 (imputed)
  Loading field 'lead_locus' as type str (imputed)
  Loading field 'region' as type str (imputed)
  Loading field 'trait_efo' as type str (imputed)
  Loading field 'trait_efo_category' as type str (imputed)
  Loading field 'locus' as type str (imputed)
  Loading field 'trait_efo_otg' as type str (imputed)
  Loading field 'study_id' as type str (imputed)
  Loading field 'trait_efo_term' as type str (imputed)
  Loading field 'trait_efo_category_otg' as type str (imputed)
2024-10-06 00:09:04.708 Hail: INFO: merging 1001 files totalling 4.4M...
2024-10-06 00:09:04.864 Hail: 

In [9]:
trait_list = ["FBS"]
res = pd.DataFrame()
for i in range(len(trait_list)):
    trait = trait_list[i]
    print(trait)
    annotate_known_loci_1Mb(trait)  

FBS


/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,,,,,,,,,,
idx,lead_locus,region,trait_efo,trait_efo_category,locus_otg,trait_efo_otg,study_id_otg,trait_efo_term,trait_efo_category_otg,locus
int32,locus<GRCh37>,interval<locus<GRCh37>>,str,str,str,str,array<str>,str,str,locus<GRCh37>
1,1:43453188,[1:42453188-1:44453188),"""EFO_0004465""","""Other measurement""","""1:42470274""","""EFO_0004612""","[""GCST90038594""]","""high density lipoprotein cholesterol measurement""","""Lipid or lipoprotein measurement""",1:42470274
1,1:43453188,[1:42453188-1:44453188),"""EFO_0004465""","""Other measurement""","""1:42470274""","""EFO_0004696""","[""GCST90038594""]","""sex hormone-binding globulin measurement""","""Other measurement""",1:42470274
1,1:43453188,[1:42453188-1:44453188),"""EFO_0004465""","""Other measurement""","""1:42470274""","""EFO_0004735""","[""GCST90038594""]","""serum alanine aminotransferase measurement""","""Liver enzyme measurement""",1:42470274


trait_efo
30
57
0
0
trait_efo_category
87
0
0
0


In [10]:
trait_list = ["FBS"]
res = pd.DataFrame()
for i in range(len(trait_list)):
    trait = trait_list[i]
    print(trait)
    annotate_known_loci(trait)  

FBS


/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,,,,,,,,,,
idx,lead_locus,region,trait_efo,trait_efo_category,locus_otg,trait_efo_otg,study_id_otg,trait_efo_term,trait_efo_category_otg,locus
int32,locus<GRCh37>,interval<locus<GRCh37>>,str,str,str,str,array<str>,str,str,locus<GRCh37>
1,1:43453188,[1:43367726-1:43460154),"""EFO_0004465""","""Other measurement""","""1:43398396""","""EFO_0004838""","[""GCST90019500""]","""calcium measurement""","""Other measurement""",1:43398396
1,1:43453188,[1:43367726-1:43460154),"""EFO_0004465""","""Other measurement""","""1:43408279""","""EFO_0004305""","[""GCST007069""]","""erythrocyte count""","""Hematological measurement""",1:43408279
1,1:43453188,[1:43367726-1:43460154),"""EFO_0004465""","""Other measurement""","""1:43408279""","""EFO_0004792""","[""GCST90014303""]","""plasminogen activator inhibitor 1 measurement""","""Other measurement""",1:43408279


trait_efo
27
60
0
0
trait_efo_category
83
4
0
0
